--- 
# Sandbox (Ambroise)
Tests for each part of this project

---

### Imports

In [19]:
%load_ext autoreload
%autoreload 2

#external libraries
import polars as pl
import pandas as pd
from datetime import date
import nest_asyncio
import os
import yfinance as yf
import datetime
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import asyncio

#internal libraries
from prompts import process_in_batches, import_data, preprocess_responses

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Importing data test

## Prompting test

### Basic sentences
Adapted from the mistral website

In [ ]:
responses = preprocess_responses()

In [ ]:
# responses.write_csv("scores.csv")

In [ ]:
# testing on half of the dataset first
titles = import_data()
titles = titles.filter(pl.col('date').lt(date(2024,1,1)))

In [ ]:
nest_asyncio.apply()
prompts = titles.select('prompts').to_series().to_list()
csv_path = "responses_2024_2025.csv"
first_write = not os.path.exists(csv_path)

async for batch in process_in_batches(prompts, batch_size=100, concurrency=5):
    batch_df = pl.DataFrame({
        "index": [r[0] for r in batch],
        "prompt": [r[1] for r in batch],
        "response": [r[2] for r in batch],
    })

    csv_string = batch_df.write_csv(separator=",", include_header=first_write)
    with open(csv_path, "a", encoding="utf-8") as f:
        f.write(csv_string)

    first_write = False
    print(f"Wrote {len(batch)} rows to {csv_path}")


In [ ]:
pl.read_csv('responses_2024_2025.csv').select(pl.col('response').str.slice(0,10))

## Testing the model

In [47]:
scores = pl.read_csv('scores_2.csv')

In [63]:
scores.head()

date,increase,decrease,diff,score
date,i64,i64,i64,f64
2017-09-29,1,0,1,1.0
2017-11-12,1,0,1,1.0
2017-11-14,1,0,1,1.0
2017-11-20,1,0,1,1.0
2017-11-23,1,0,1,1.0


In [49]:
scores = pl.read_csv(
  "scores_2.csv",
  dtypes={"date": pl.Date}
)


/var/folders/zq/wrfz6jts6k73jk85nmz4ybrh0000gn/T/ipykernel_66388/2113586872.py:1: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  scores = pl.read_csv(


In [62]:
# drop when score is not NaN
scores = scores.filter(pl.col('score').is_not_nan())

In [64]:
min_date = scores["date"].min()
max_date = scores["date"].max()

start_str = min_date.strftime("%Y-%m-%d")
end_str   = (max_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

df_pd = yf.download(
    tickers="BTC-USD",
    start=start_str,
    end=end_str,
    interval="1d",
    progress=False
)


if isinstance(df_pd.columns, pd.MultiIndex):
    df_pd.columns = df_pd.columns.get_level_values(0)

df_pd = df_pd.reset_index()
df_price = (
    pl.from_pandas(df_pd)
      .select([
         pl.col("Date").alias("date").cast(pl.Date),
         pl.col("Close").alias("close")
      ])
)

print(df_price.head())


shape: (5, 2)
┌────────────┬─────────────┐
│ date       ┆ close       │
│ ---        ┆ ---         │
│ date       ┆ f64         │
╞════════════╪═════════════╡
│ 2017-09-29 ┆ 4163.069824 │
│ 2017-09-30 ┆ 4338.709961 │
│ 2017-10-01 ┆ 4403.740234 │
│ 2017-10-02 ┆ 4409.319824 │
│ 2017-10-03 ┆ 4317.47998  │
└────────────┴─────────────┘


In [65]:
# for each date, if next date is higher : add 1, else 0
df_price = df_price.with_columns([
    (pl.col("close").shift(-1) > pl.col("close")).cast(pl.Int8).alias("actual")
])

In [66]:
df = scores.join(df_price.select(["date","actual"]), on="date", how="inner")

In [67]:
df = df.with_columns(
    (pl.col("score") > 0).cast(pl.Int8).alias("predicted")
)

In [68]:
print(df.head(50))

shape: (50, 7)
┌────────────┬──────────┬──────────┬──────┬───────┬────────┬───────────┐
│ date       ┆ increase ┆ decrease ┆ diff ┆ score ┆ actual ┆ predicted │
│ ---        ┆ ---      ┆ ---      ┆ ---  ┆ ---   ┆ ---    ┆ ---       │
│ date       ┆ i64      ┆ i64      ┆ i64  ┆ f64   ┆ i8     ┆ i8        │
╞════════════╪══════════╪══════════╪══════╪═══════╪════════╪═══════════╡
│ 2017-09-29 ┆ 1        ┆ 0        ┆ 1    ┆ 1.0   ┆ 1      ┆ 1         │
│ 2017-11-12 ┆ 1        ┆ 0        ┆ 1    ┆ 1.0   ┆ 1      ┆ 1         │
│ 2017-11-14 ┆ 1        ┆ 0        ┆ 1    ┆ 1.0   ┆ 1      ┆ 1         │
│ 2017-11-20 ┆ 1        ┆ 0        ┆ 1    ┆ 1.0   ┆ 0      ┆ 1         │
│ 2017-11-23 ┆ 1        ┆ 0        ┆ 1    ┆ 1.0   ┆ 1      ┆ 1         │
│ …          ┆ …        ┆ …        ┆ …    ┆ …     ┆ …      ┆ …         │
│ 2018-06-15 ┆ 1        ┆ 0        ┆ 1    ┆ 1.0   ┆ 1      ┆ 1         │
│ 2018-06-22 ┆ 0        ┆ 1        ┆ -1   ┆ -1.0  ┆ 1      ┆ 0         │
│ 2018-06-26 ┆ 0        ┆ 1        ┆

In [69]:
df_clean = df.filter(
    pl.col("actual").is_not_null() &
    pl.col("predicted").is_not_null()
)

pdf = df_clean.select(["actual", "predicted"]).to_pandas()

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

acc  = accuracy_score(pdf.actual, pdf.predicted)
cm   = confusion_matrix(pdf.actual, pdf.predicted)
prec = precision_score(pdf.actual, pdf.predicted)
rec  = recall_score(pdf.actual, pdf.predicted)

print(f"Accuracy       : {acc:.3f}")
print(f"Confusion mat. :\n{cm}")
print(f"Precision      : {prec:.3f}")
print(f"Recall         : {rec:.3f}")


Accuracy       : 0.487
Confusion mat. :
[[161 601]
 [199 598]]
Precision      : 0.499
Recall         : 0.750


In [70]:
pdf

,actual,predicted
0,1,1
1,1,1
2,1,1
3,0,1
4,1,1
...,...,...
1554,0,1
1555,1,1
1556,0,1
1557,0,1


## New prompts with differents dates ranges.

In [ ]:
import os
import nest_asyncio
import polars as pl
from datetime import date

# --- Paramètres à ajuster pour chaque période ---
start_dt = date(2023, 1,  1)
end_dt   = date(2025, 1,  1)
csv_path = "responses_2023_2025.csv"
# -----------------------------------------------

# Charger tout et filtrer
titles = import_data().with_row_count("index")
subset = titles.filter(
    (pl.col("date") >= start_dt) &
    (pl.col("date") <  end_dt)
)
prompts = subset["prompts"].to_list()



In [20]:
from tqdm.asyncio import tqdm_asyncio

# Préparer le CSV
if os.path.exists(csv_path):
    os.remove(csv_path)
first_write = True

# Appliquer nest_asyncio
nest_asyncio.apply()

# Boucle asynchrone en batch with tqdm
async for batch in tqdm_asyncio(process_in_batches(prompts, batch_size=200, concurrency=5), desc="Processing batches"):
    batch_df = pl.DataFrame({
        "index":    [r[0] for r in batch],
        "prompt":   [r[1] for r in batch],
        "response": [r[2] for r in batch],
    })

    csv_str = batch_df.write_csv(separator=",", include_header=first_write)
    with open(csv_path, "a", encoding="utf-8") as f:
        f.write(csv_str)

    first_write = False
    print(f"--------Wrote {len(batch)} rows to {csv_path}--------")


Processing batches: 0it [00:00, ?it/s]

Processing prompts 0 to 199...
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] R

Processing batches: 1it [01:48, 108.23s/it]

--------Wrote 200 rows to responses_2023_2025.csv--------
Processing prompts 200 to 399...
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, ret

Processing batches: 2it [03:27, 102.72s/it]

...

--------Wrote 200 rows to responses_2023_2025.csv--------
Processing prompts 14000 to 14199...
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit,

Processing batches: 71it [3:28:34, 234.28s/it]

--------Wrote 200 rows to responses_2023_2025.csv--------
Processing prompts 14200 to 14354...
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 3/4 in 4.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 1/4 in 1.0s…
[429] Rate limit, retry 2/4 in 2.0s…
[429] Rate limit, retry 4/4 in 8.0s…
[429] Rate limit,

Processing batches: 72it [3:31:33, 176.30s/it]

--------Wrote 155 rows to responses_2023_2025.csv--------


## Aggregating the results and saving in a csv file

## score dataframe

In [ ]:
scores_2 = preprocess_responses(['responses_2017_2019.csv', 'responses_2019_2021.csv', 'responses_2021_2023.csv', 'responses_2023_2025.csv'])